In [1]:
import os
import json
import pandas as pd
from random import randrange
import random

The history saving thread hit an unexpected error (OperationalError('no such table: history')).History will not be written to the database.


In [2]:
# create folder for each dataset first    

In [3]:
def save_json(content, save_path):
    with open(save_path, 'w') as f:
        f.write(json.dumps(content))
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]

In [4]:
# TVQA

In [5]:
#set tvqa video, input json, eval json folder
tvqa_folder = "/home/hlpark/TVQA/video/video_files/"
if not os.path.exists(tvqa_folder):
    os.makedirs(tvqa_folder)

eval_folder = "/home/hlpark/REDUCE/REDUCE_benchmarks/SeViLA/sevila_data/tvqa_evaluation_json"
if not os.path.exists(eval_folder):
    os.makedirs(eval_folder)

vid_json_folder = "/home/hlpark/REDUCE/REDUCE_benchmarks/SeViLA/sevila_data/tvqa"
if not os.path.exists(vid_json_folder):
    os.makedirs(vid_json_folder)

In [6]:
vid_json_folder = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa"
clip_pred_med = load_jsonl(f'{vid_json_folder}/five_labeled_pred_med_from_gt_vid_dict.json')

In [7]:
train_path = '/home/hlpark/shared/TVQA/tvqa_qa_release/tvqa_train.jsonl'
val_path = '/home/hlpark/shared/TVQA/tvqa_qa_release/tvqa_val.jsonl'
test_path = '/home/hlpark/shared/TVQA/tvqa_qa_release/tvqa_test_public.jsonl'

#below predicions were on TVQA dataset with audio
#original before finetuning
hirest_prediction_test_path_original_path = '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_test_moment_retrieval_BEST_with_audio.json'
hirest_prediction_val_path_original_path = '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_val_moment_retrieval_BEST_with_audio.json'
#C : TVQA with audio
hirest_prediction_test_path_finetuned_on_visual_from_med_path = '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_on_visual_med/all_data_test_moment_retrieval_BEST_with_audio.json'
hirest_prediction_val_path_finetuned_on_visual_from_med_path = '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_on_visual_med/all_data_val_moment_retrieval_BEST_with_audio.json'
#D : TVQA with audio
hirest_prediction_test_path_finetuned_on_visual_from_nonmed_path = '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_on_visual_nonmed/all_data_test_moment_retrieval_BEST_with_audio.json'
hirest_prediction_val_path_finetuned_on_visual_from_nonmed_path = '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_on_visual_nonmed/all_data_val_moment_retrieval_BEST_with_audio.json'
#F : TVQA with audio
hirest_prediction_test_path_finetuned_on_full_without_audio_path = '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_on_full_without_audio/all_data_test_moment_retrieval_BEST_with_audio.json'
hirest_prediction_val_path_finetuned_on_full_without_audio_path  = '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_finetuned_on_full_without_audio/all_data_val_moment_retrieval_BEST_with_audio.json'

In [8]:
train = load_jsonl(train_path)
val = load_jsonl(val_path)
test = load_jsonl(test_path)

In [9]:
prediction_set = [[json.load(open(hirest_prediction_test_path_original_path)), json.load(open(hirest_prediction_val_path_original_path))],
                  [json.load(open(hirest_prediction_test_path_finetuned_on_visual_from_med_path)), json.load(open(hirest_prediction_val_path_finetuned_on_visual_from_med_path))],
                  [json.load(open(hirest_prediction_test_path_finetuned_on_visual_from_nonmed_path)), json.load(open(hirest_prediction_val_path_finetuned_on_visual_from_nonmed_path))],
                  [json.load(open(hirest_prediction_test_path_finetuned_on_full_without_audio_path)), json.load(open(hirest_prediction_val_path_finetuned_on_full_without_audio_path))]]

In [10]:
tvqa_hirest_folder_list = ['/home/hlpark/REDUCE/REDUCE_benchmarks/SeViLA/sevila_data/hirest_tvqa_original_pred',
                           '/home/hlpark/REDUCE/REDUCE_benchmarks/SeViLA/sevila_data/hirest_tvqa_finetuned_on_visual_med',
                           '/home/hlpark/REDUCE/REDUCE_benchmarks/SeViLA/sevila_data/hirest_tvqa_finetuned_on_visual_nonmed',
                           '/home/hlpark/REDUCE/REDUCE_benchmarks/SeViLA/sevila_data/hirest_tvqa_finetuned_on_full_without_audio']

In [12]:
medical_val_set = {}
nonmedical_val_set = {}
c, d = 0, 0
for i, qa in enumerate(val):
    for qc in clip_pred_med[0][qa['vid_name']]:
        if len(qc) != len(set(qc)):
            print(qc)
        if qa['q'] not in qc:
            continue
        if qc[qa['q']] == "med":
            medical_val_set[qa['q'] + qa['vid_name']] = 1
            c += 1
        elif qc[qa['q']] == "nonmed":
            nonmedical_val_set[qa['q'] + qa['vid_name']] = 1
            d+=1


random.seed(91)
med_keys = list(medical_val_set)
nonmed_keys = list(nonmedical_val_set)
print(len(medical_val_set), len(nonmedical_val_set))
print(len(list(range(0, len(medical_val_set)))))
is_1000_orig = False
is_700 = True
if is_1000_orig:
    med_idx = random.sample(list(range(0, len(medical_val_set))), k = 1000)
    if len(med_idx) != len(set(med_idx)):
        print("duplicate")
    non_med_idx = random.sample(list(range(0, len(nonmedical_val_set))), k = 1000)
    if len(non_med_idx) != len(set(non_med_idx)):
        print("duplicate")
elif is_700 == False:
    med_idx = random.sample(list(range(0, len(medical_val_set))), k = 1000)

    med_new_idx = [x for x in list(range(0, len(medical_val_set))) if x not in med_idx]
    med_idx = med_new_idx[:1000]
    if len(med_idx) != len(set(med_idx)):
        print("duplicate")
    non_med_idx = random.sample(list(range(0, len(nonmedical_val_set))), k = 1000)
    non_med_new_idx = [x for x in list(range(0, len(nonmedical_val_set))) if x not in non_med_idx]
    non_med_idx = non_med_new_idx[:1000]
    if len(non_med_idx) != len(set(non_med_idx)):
        print("duplicate")
else:
    med_idx = random.sample(list(range(0, len(medical_val_set))), k = 1000)

    med_new_idx = [x for x in list(range(0, len(medical_val_set))) if x not in med_idx]
    med_idx = med_new_idx[1000:]
    if len(med_idx) != len(set(med_idx)):
        print("duplicate")
    non_med_idx = random.sample(list(range(0, len(nonmedical_val_set))), k = 1000)
    non_med_new_idx = [x for x in list(range(0, len(nonmedical_val_set))) if x not in non_med_idx]
    non_med_idx = non_med_new_idx[1000:1000+len(med_idx)]
    if len(non_med_idx) != len(set(non_med_idx)):
        print("duplicate")
medical_val_1000_sample = {}
nonmedical_val_1000_sample = {}

for idx in med_idx:
    medical_val_1000_sample[med_keys[idx]] = medical_val_set[med_keys[idx]]

for idx in non_med_idx:
    nonmedical_val_1000_sample[nonmed_keys[idx]] = nonmedical_val_set[nonmed_keys[idx]]

assert len(medical_val_1000_sample) == len(medical_val_1000_sample)
assert len(nonmedical_val_1000_sample) == len(nonmedical_val_1000_sample)
print(len(medical_val_1000_sample))
print(len(nonmedical_val_1000_sample))
# for i, x in enumerate(medical_val_1000_sample['query']):
#     if x in nonmedical_val_1000_sample['query']:
#         print(medical_val_1000_sample['video'][i])
#         for j, y in enumerate(nonmedical_val_1000_sample['query']):
#             if y == x:
#                 print(nonmedical_val_1000_sample['video'][j])

2704 12548
2704
704
704


In [13]:
video_names = {}

for idx, pred in enumerate(prediction_set):
    pred_test, pred_val = pred[0], pred[1]
    val_dict, partial_val_dict = [], []
    dupq_cnt_test, dupq_cnt_val = 0, 0
    for i, qa in enumerate(val):
        qa_dict = {}
        
        if qa['vid_name'] not in video_names:
            video_names[qa['vid_name']] = []
        qa_dict['video'] = qa['vid_name']
        qa_dict['num_option'] = 5
        qa_dict['qid'] = 'TVQA_' + str(i)
        for j in range(5):
            qa_dict['a{}'.format(str(j))] = qa['a{}'.format(str(j))]
        qa_dict['answer'] = qa['answer_idx']
        qa_dict['question'] = qa['q']
        
        
        if qa['q'] in pred_val:
            # print(pred_val[qa['q']], qa['q'], qa['vid_name'])
            if qa['vid_name'] + ".mp4" not in pred_val[qa['q']]:
                dupq_cnt_val += 1
                continue
            qa_dict['start'] = pred_val[qa['q']][qa['vid_name'] + ".mp4"]['bounds'][0]
            qa_dict['end'] = pred_val[qa['q']][qa['vid_name'] + ".mp4"]['bounds'][1]
        else:
            print("ERROR ", qa['q'])
            continue
        if qa['q'] + qa['vid_name']  in medical_val_1000_sample:
            partial_val_dict.append(qa_dict)
        elif qa['q'] + qa['vid_name'] in nonmedical_val_1000_sample:
            partial_val_dict.append(qa_dict)
        val_dict.append(qa_dict)

    test_dict, partial_test_dict = [], []
    for i, qa in enumerate(test):
        qa_dict = {}
        if qa['vid_name'] not in video_names:
            video_names[qa['vid_name']] = []
        qa_dict['video'] = qa['vid_name']
        qa_dict['num_option'] = 5
        qa_dict['qid'] = 'TVQA_' + str(i)
        for j in range(5):
            qa_dict['a{}'.format(str(j))] = qa['a{}'.format(str(j))]
        qa_dict['answer'] = randrange(5)
        qa_dict['question'] = qa['q']
        qa_dict['start'] = qa['ts'].split('-')[0]
        qa_dict['end'] = qa['ts'].split('-')[1]
        if qa['q'] in pred_test:
            if qa['vid_name'] + ".mp4" not in pred_test[qa['q']]:
                dupq_cnt_test += 1
                continue
            qa_dict['start'] = pred_test[qa['q']][qa['vid_name'] + ".mp4"]['bounds'][0]
            qa_dict['end'] = pred_test[qa['q']][qa['vid_name'] + ".mp4"]['bounds'][1]
        else:
            print("ERROR ", qa['q'])
            continue
        test_dict.append(qa_dict)
    print(len(partial_val_dict))
    print(dupq_cnt_val, dupq_cnt_test)
    if not os.path.exists(tvqa_hirest_folder_list[idx]):
        os.makedirs(tvqa_hirest_folder_list[idx])
    save_json(val_dict, f'{tvqa_hirest_folder_list[idx]}/val_hirest_gt.json')
    if is_1000_orig:
        save_json(partial_val_dict, f'{tvqa_hirest_folder_list[idx]}/partial_val_hirest_gt.json')
    else:
        if is_700 == False:
            print("pt2")
            #save_json(partial_val_dict, f'{tvqa_hirest_folder_list[idx]}/partial_val_hirest_gt_pt2.json')
        else:
            save_json(partial_val_dict, f'{tvqa_hirest_folder_list[idx]}/partial_val_hirest_gt_pt3.json')
    save_json(test_dict, f'{tvqa_hirest_folder_list[idx]}/test_hirest_gt.json')

1406
8 5
1406
8 5
1406
8 5
1406
8 5
